In [15]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
from scipy.special import gamma, loggamma, factorial
import scipy.stats
from scipy.interpolate import interp1d
from matplotlib.ticker import AutoMinorLocator 
from matplotlib import rc, font_manager
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from mycolorpy import colorlist as mcp
from matplotlib import legend_handler
from matplotlib_scalebar.scalebar import ScaleBar
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
import collections
import matplotlib
import shapely.geometry
from shapely.geometry import Point
import shapely.ops 
from geopy.geocoders import Nominatim
import geopandas as gpd
import mapclassify
import jenkspy
import contextily as cx
import os
import rioxarray as rx
import pyreadr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from pysal.lib import weights
from libpysal.io import open as psopen
# import plots
import seaborn as sns
from datetime import datetime, timedelta
import jenkspy
import warnings
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.formula.api as smf
warnings.filterwarnings("ignore")


In [55]:
wd = '/Users/carmen/Library/CloudStorage/OneDrive-TheUniversityofLiverpool/research/RECAST/latin-mobility-covid-local-files'

country = 'Chile'

if country=='Argentina':
    country_short = 'ARG'
    country_code = 'AR'
elif country == 'Chile':
    country_short = 'CHL'
    country_code = 'CL'
elif country == 'Colombia':
    country_short = 'COL'
    country_code = 'CO'
elif country == 'Mexico':
    country_short = 'MEX'
    country_code = 'MX'

In [56]:
category = 'density'

In [57]:
df_trend = pd.read_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/time_series_trend.csv').drop('Unnamed: 0', axis=1)
df_trend = df_trend.T

In [58]:
n_class = np.sqrt(len(df_trend.columns) + 1)
O = []
D = []
time = []
y = []
for i in range(len(df_trend.columns)):
    for j in range(len(df_trend)):
        O.append(int(i/n_class))
        D.append(i%n_class)
        time.append(j)
        y.append(df_trend.iloc[j, i])
        

data = pd.DataFrame({'O': O, 'D': D, 'y': y, 'time': time})

In [59]:
data.to_csv(wd + '/data/outputs/' + country_short + '/mov-analysis/by-' + category + '/data_trend.csv')



In [60]:
data

,O,D,y,time
0,0,0.000000,-30.951603,0
1,0,0.000000,-30.631122,1
2,0,0.000000,-30.310641,2
3,0,0.000000,-29.990159,3
4,0,0.000000,-29.669678,4
...,...,...,...,...
2620,4,3.603922,-29.888991,100
2621,4,3.603922,-29.680786,101
2622,4,3.603922,-29.472581,102
2623,4,3.603922,-29.264376,103


### A few tests using python packages for mixed lm

In [10]:
md = smf.mixedlm("y ~ time", data, groups=data["O"], re_formula="~time")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

             Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   y         
No. Observations:    2664      Method:               REML      
No. Groups:          5         Scale:                42.5330   
Min. group size:     444       Log-Likelihood:       -8795.4907
Max. group size:     555       Converged:            Yes       
Mean group size:     532.8                                     
---------------------------------------------------------------
                  Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
---------------------------------------------------------------
Intercept        -26.084    2.199 -11.860 0.000 -30.394 -21.773
time               0.300    0.013  22.676 0.000   0.274   0.326
Group Var         23.866    2.629                              
Group x time Cov  -0.085    0.013                              
time Var           0.001    0.000                              

